#  Libraries

In [1]:
import os, csv, re, glob, math, collections, sys, codecs
import xml.etree.ElementTree as ET
import pandas as pd
from matplotlib import pylab as plt
import matplotlib.pyplot as plot
import seaborn as sns
import numpy as np
from numpy import *
from scipy import signal
from scipy.signal import find_peaks
from scipy import stats
from io import BytesIO
from colorama import *

# Profile gathering

## Experion files import - XML + CSV

In [2]:
full_xml_path = os.path.abspath(os.path.join(".", "run3.xml"))
full_csv_path = os.path.abspath(os.path.join(".", "run_output3.csv"))


tree = ET.parse(full_xml_path)
root = tree.getroot()

file_header = ['sample']
file_header_vals = []
sample_well_info = []

*   Set File Header

In [3]:
for i in range(475, 869):
    file_header.append(str(round((0.00042006 * (i * i) - 0.28216 * i + 49.225), 1)))
    file_header_vals.append(str(round((0.00042006 * (i * i) - 0.28216 * i + 49.225), 1)))

*   get all well id's and sample names from file

In [4]:
for sample in root.findall('Results'):
    # write sample name
    well_id = sample.find('WellId').text

    if well_id != '0':
        sample_name = sample.find('SampleName').text
        sample_well_info.append({'sample': sample_name, 'well_id': well_id})

*   create file with headers and data

In [5]:
with open(full_csv_path, 'w', newline='') as outcsv:
    writer = csv.DictWriter(outcsv, fieldnames = file_header)
    writer.writeheader()
    xmlstr = ET.tostring(root, encoding='utf8', method='xml').decode("utf-8")

    for elem in sample_well_info:
        
        idx = 0
        row = {'sample': elem['sample']}
        regex = '<RawDataPoints><WellId>%s<\/WellId><Signal>(.*?)<\/Signal><\/RawDataPoints>'% elem['well_id']
        raw_data_points = re.findall(regex, xmlstr)
        
        # write data for each column
        for count, raw_data_point in enumerate(raw_data_points, start=0):
            if 869 >= count >= 476:
                row[file_header_vals[idx]] = raw_data_point
                idx+=1
            
            if count > 869:
                break
        
        writer.writerow(row)

## Plot profiles

* transforming data to seaborn plot table

In [6]:
file1 = open('run_output.csv', 'r')
Lines = file1.readlines()

outfile = open('profile_plot.csv', 'w')
outfile.write('Sample,Molecular Weight,Fluorescence,\n')

header = Lines[0].strip().split(',')
mws = header[1:]

for line in Lines[1:]:
    line_info = line.strip().split(',')

    id = line_info[0]
    fl = line_info[1:]

    ix = 0
    for mw in mws:
        outfile.write(id + ','+ mws[ix]+','+fl[ix] +'\n')
        ix += 1

outfile.close()

FileNotFoundError: [Errno 2] No such file or directory: 'run_output.csv'

* seaborn line plot

In [7]:
sb_plot = pd.read_csv("profile_plot.csv", header=0) #Dataset for ploting profiles using seaborn
sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(18, 8))

profile_plot = sns.lineplot(x="Molecular Weight", y="Fluorescence",
             hue="Sample", data=sb_plot);

profile_plot.set(xlim=(8, 122));

FileNotFoundError: [Errno 2] File profile_plot.csv does not exist: 'profile_plot.csv'

In [8]:
df1 = pd.read_csv('chip1_2.csv')
df2 = pd.read_csv('chip3.csv')
df3 = pd.read_csv('chip4.csv')
df4 = pd.read_csv('chip5.csv')
df5 = pd.read_csv('chip6.csv')

finaldf = pd.concat([df1, df2, df3,df4,df5], axis=0, join='inner')
finaldf.to_csv('finaldf.csv')

# Peak area calculation

In [9]:
import plotly.graph_objects as go
import pandas as pd
from scipy.signal import find_peaks

In [10]:
x = electrocardiogram()[17000:18000]
peaks, properties = find_peaks(x, prominence=1, width=20)
properties["prominences"], properties["widths"]
(array([1.495, 2.3  ]), array([36.93773946, 39.32723577]))
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.vlines(x=peaks, ymin=x[peaks] - properties["prominences"],
           ymax = x[peaks], color = "C1")
plt.hlines(y=properties["width_heights"], xmin=properties["left_ips"],
           xmax=properties["right_ips"], color = "C1")
plt.show()

NameError: name 'electrocardiogram' is not defined

In [11]:
x1 = pd.read_csv("profile_plot.csv", header=0)
x1.head()
s1 = x1.loc[x1["Sample"] == "S1"]
s1.head()
# x2 = x1.drop(columns=[0], index=0)
# x2.head()
# x = x2.T
# x.head()  

FileNotFoundError: [Errno 2] File profile_plot.csv does not exist: 'profile_plot.csv'

In [12]:
x1 = pd.read_csv("finaldf.csv", header=None)
x1.head()
# x2 = x1.drop(columns=[0], index=0)
# x2.head()
x = x1.T
x.head()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,NaN,0,1,2,3,4,5,6,7,8,...,2,3,0,1,2,3,4,5,6,7
1,Sample,D2898,D2898,D2696,D2696,D2700,D2700,D2829,D2829,D2828,...,D2857,D2857,D2835,D2835,D2902,D2902,D2903,D2903,D2904,D2904
2,10,3.19908,5.13934,45.7251,33.5953,2.59931,2.7342,84.6405,100.915,37.6454,...,-7.81932,-1.25854,-6.29126,-4.08995,-23.4868,-31.4139,-50.0775,-59.4309,-36.1302,-41.4568
3,10.1,0.216139,3.01306,42.174,30.5372,1.96629,3.49847,79.6725,95.1979,35.2528,...,2.38565,-1.32897,-7.26916,-5.55318,-24.8028,-33.2384,-50.6457,-59.402,-36.0183,-39.6744
4,10.2,-2.99206,1.55404,37.7835,27.0123,1.18924,3.94283,71.9109,84.7288,31.3269,...,25.0425,-1.62588,-7.55755,-6.87381,-25.355,-34.8513,-51.2941,-59.5966,-36.9937,-39.0833


In [13]:
i = 0
fl = np.array(x.iloc[:, i: i + 1]).squeeze()

In [14]:
peaks, _ = find_peaks(fl)
plt.plot(fl)
# plt.plot(peaks, fl[peaks], "fl")
plt.show()

ValueError: could not convert string to float: 'Sample'

In [15]:
fluorescence = s1['Fluorescence']
fluorescence.head()

NameError: name 's1' is not defined

In [16]:
milk_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/monthly-milk-production-pounds.csv')
milk_data.head()
time_series = milk_data['Monthly milk production (pounds per cow)']
time_series.head()

0    589
1    561
2    640
3    656
4    727
Name: Monthly milk production (pounds per cow), dtype: int64

In [17]:
from scipy.signal import find_peaks, peak_prominences
peaks, _ = find_peaks(fl, height=25)
prominences = peak_prominences(fl, peaks)[0]
prominences

ValueError: could not convert string to float: 'Sample'

In [18]:
indices = find_peaks(fl, height=25)[0]
indices

ValueError: could not convert string to float: 'Sample'

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=fl,
    mode='lines+markers',
    name='Original Plot'
))


In [20]:
fig.add_trace(go.Scatter(
    x=indices,
    y=[fl[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))

fig.show()

TypeError: 'function' object is not iterable

In [21]:
peaks, properties = find_peaks(x, prominence=1, width=20)

ValueError: could not convert string to float: 'D2904'

In [22]:
properties["prominences"], properties["widths"]

NameError: name 'properties' is not defined

In [23]:
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.vlines(x=peaks, ymin=x[peaks] - properties["prominences"],
           ymax = x[peaks], color = "C1")
plt.hlines(y=properties["width_heights"], xmin=properties["left_ips"],
           xmax=properties["right_ips"], color = "C1")
plt.show()

ValueError: could not convert string to float: 'Sample'

In [24]:
proteins_df = pd.read_csv("finaldf.csv", header=0)
proteins_df_features = proteins_df.drop(columns=['Sample'])
proteins_features = proteins_df_features.columns
peaks_data_dict = {}
all_groups = []
samples_qtd = proteins_features.shape[0]

In [67]:
for i in range(0, samples_qtd):
    line = proteins_df_features.iloc[i]
    peaks, properties = find_peaks(line, prominence=25, width=1, height=125)

    plt.figure(figsize=(16, 8))
    plt.plot(proteins_features, line)
    plt.ylabel('Fluorescence Signal')
    plt.xlabel('Molecular Weight (kDa)')
    np.diff(peaks)
    plt.plot(peaks, line[peaks], "x")
    plt.vlines(x=peaks, ymin=line[peaks] - properties["prominences"], ymax=line[peaks], color="C1")
    plt.hlines(y=properties["width_heights"], xmin=properties["left_ips"], xmax=properties["right_ips"], color="C1")
    plt.xticks(rotation=45)

    # plt.savefig('results/peaks/' + proteins_df['Sample'][i] + '_peaks.pdf', format='pdf', bbox_inches='tight')
    plt.close()

    # Calculando áreas sobre os picos
    cuts = (peaks[1:] + peaks[:-1]) // 2
    cuts = np.insert(cuts, [0, cuts.size], [0, len(proteins_features)])
    peak_begins = np.zeros_like(peaks)
    peak_ends = np.zeros_like(peaks)
    areas = []
    peak_proteins_features = []

    for j in range(peaks.size):
        peak_protein_group = proteins_features[peaks[j]]
        peak_value = line.values[peaks[j]]
        y_cut = line.values[cuts[j]:cuts[j + 1]]
        baseline = np.median(y_cut)
        large = np.where(y_cut > 0.5 * (peak_value + baseline))[0]
        peak_begins[j] = large.min() + cuts[j]
        peak_ends[j] = large.max() + cuts[j]
        peak_proteins_features.append(peak_protein_group)
        areas.append(np.sum(line.values[peak_begins[j]:peak_ends[j]] - baseline))

    all_groups.append(peak_proteins_features)

    peaks_data_partial = {}
    for k in range(0, len(peak_proteins_features)):
        peaks_data_partial[peak_proteins_features[k]] = [properties['peak_heights'][k], properties['prominences'][k],
                                                       properties['width_heights'][k], areas[k]]

    peaks_data_dict[proteins_df['Sample'][i]] = peaks_data_partial

IndexError: single positional indexer is out-of-bounds

In [ ]:
all_groups = list(set([item for sublist in all_groups for item in sublist]))

In [ ]:
peaks_data = []
for sample in peaks_data_dict.keys():
    curr_sample_values = []

    for group in all_groups:
        if group in peaks_data_dict[sample].keys():
            sample_data = peaks_data_dict[sample][group]
            sample_data.insert(0, 1)
            curr_sample_values.append(sample_data)
        else:
            curr_sample_values.append([0, 0, 0, 0, 0])

    curr_sample_values = [item for sublist in curr_sample_values for item in sublist]
    curr_sample_values.insert(0, sample)
    peaks_data.append(curr_sample_values)

In [ ]:
all_groups.sort(key=lambda x : int(x.split('-')[0]))

In [ ]:
peaks_cols = ["sample"]
for group in all_groups:
    peaks_cols.append("p_" + group)
    peaks_cols.append("p_" + group + "_height")
    peaks_cols.append("p_" + group + "_prominence")
    peaks_cols.append("p_" + group + "_width")
    peaks_cols.append("p_" + group + "_area")

In [ ]:
peaks_df = pd.DataFrame(peaks_data, columns=peaks_cols)
peaks_df = pd.merge(peaks_df, proteins_df[['sample', 'oral_diagnosis']], on='sample', how='left')

In [ ]:
peaks_df.to_csv('results/peaks_dataset.csv', index=False)
peaks_df.head()